In [1]:
import pandas, re

In [2]:
with open('./ppi_final.txt', 'r') as infile:
    data = pandas.read_csv(infile, delimiter = '\t', header = None)

In [3]:
data

,0,1
0,lacZ,lacZ
1,"{lacZ, lacZ}","{lacZ, lacZ}"
2,lacA,lacA
3,lacA,"{lacA, lacA}"
4,crp,crp
5,"lacZ, lacZ, lacZ",lacZ


In [4]:
for i in data.index:
    # LHS
    agents = (data.iloc[i, 0] + ', ' + data.iloc[i, 1])
#     print(agents)
    # find complexes in LHS
    complexes = [(m.start()+1, m.end()-1) for m in re.finditer(r'{[A-Za-z-_, ]+}', agents)]
    monomers = [(m.start(), m.end()) for m in re.finditer(r'[A-Za-z-_]+', agents)]
#     print(monomers, complexes)
    
    names = agents.split(', ')
#     print(monomers)
    
    LHS = []
    for molecule in names:
        if molecule[0] == '{': # we are dealing with the first monomer of a complex
            LHS.append('prot(name = \'{:s}\', up = {{:s}}, dw = {{:s}})'.format(molecule[1:]))
        elif molecule[-1] == '}': # we are dealing with the last monomer of a complex
            LHS.append('prot(name = \'{:s}\', up = {{:s}}, dw = {{:s}})'.format(molecule[:-1]))
        else: # we have a monomer
            LHS.append('prot(name = \'{:s}\', up = None, dw = None)'.format(molecule))
            
    if complexes == []:
        LHS = ' + '.join(LHS)
    else: # look for where starts and ends a complex in LHS
        positions = []
        for cplx_pos in reversed(complexes):
            pos_i = None
            pos_f = None
            for index, kmer_pos in enumerate(monomers):
                
                if cplx_pos[0] == kmer_pos[0]:
                    pos_i = index
                if cplx_pos[1] == kmer_pos[1]:
                    pos_f = index
                    positions.append((pos_i, pos_f))
                    break
                    
        # join complexes following start and end positions
        tmp = LHS
        sub_LHS = []
        for pos in positions:
#             print(pos[0], pos[1])
            count_monomers = len(LHS[pos[0]:pos[1]+1])
            up = list(range(count_monomers))
            dw = [up[-1]] + up[:-1]
            up[0] = 'None'
            dw[1] = 'None'
            
            for index in range(count_monomers):
#                 print(up, LHS[pos[0]])
#                 print(dw, LHS[pos[1]])
#                 print()
                
                LHS[pos[0]] = LHS[pos[0]].format(str(dw[index]), str(up[index]))
                LHS[pos[1]] = LHS[pos[1]].format(str(up[index]), str(dw[index]))
    
            LHS[pos[0]] = ' % '.join(LHS[pos[0]:pos[1]+1])
            LHS.pop(pos[1])
        
        LHS = ' + '.join(LHS)
    
#     print(LHS)
#     print()
    
    # RHS
    monomers = (data.iloc[i, 0] + ', ' + data.iloc[i, 1]).replace('{', '').replace('}', '')
    monomers = monomers.split(', ')
    
    RHS = []
    count = list(range(len(monomers)))
    up = count
    dw = [count[-1]] + count[:-1]
    
    up[0] = 'None'
    dw[1] = 'None'

    for index, molecule in enumerate(monomers):
        RHS.append('prot(name = \'{:s}\', up = {:s}, dw = {:s})'.format(molecule, str(up[index]), str(dw[index])))
    
    RHS = ' % '.join(sorted(RHS))
#     print(RHS)
#     print()
    
    # complete rule
    name = 'complex_assembly_rule_' + str(i)
    print('Rule(\'{:s}\', \n{:s} | \n{:s}, \nParameter(\'fwd_{:s}\', 0), Parameter(\'rvs_{:s}\', 0))'. \
          format(name, LHS, RHS, name, name).replace('-', '_'))
    print()

Rule('complex_assembly_rule_0', 
prot(name = 'lacZ', up = None, dw = None) + prot(name = 'lacZ', up = None, dw = None) | 
prot(name = 'lacZ', up = 1, dw = None) % prot(name = 'lacZ', up = None, dw = 1), 
Parameter('fwd_complex_assembly_rule_0', 0), Parameter('rvs_complex_assembly_rule_0', 0))

Rule('complex_assembly_rule_1', 
prot(name = 'lacZ', up = 1, dw = None) % prot(name = 'lacZ', up = None, dw = 1) + prot(name = 'lacZ', up = 1, dw = None) % prot(name = 'lacZ', up = None, dw = 1) | 
prot(name = 'lacZ', up = 1, dw = None) % prot(name = 'lacZ', up = 2, dw = 1) % prot(name = 'lacZ', up = 3, dw = 2) % prot(name = 'lacZ', up = None, dw = 3), 
Parameter('fwd_complex_assembly_rule_1', 0), Parameter('rvs_complex_assembly_rule_1', 0))

Rule('complex_assembly_rule_2', 
prot(name = 'lacA', up = None, dw = None) + prot(name = 'lacA', up = None, dw = None) | 
prot(name = 'lacA', up = 1, dw = None) % prot(name = 'lacA', up = None, dw = 1), 
Parameter('fwd_complex_assembly_rule_2', 0), Parameter